In [1]:
import augur
from augur.PiperReader import PiperMail
from sqlalchemy.ext.declarative import declarative_base
# import everything from githubapi.py and ghtorrent.py so we can
# just copy and paste our function later
import json
import pandas as pd
from perceval.backends.core.pipermail import Pipermail,PipermailList
from perceval.backends.core.mbox import MBox
import perceval
import os, os.path
import sqlalchemy as s
from sqlalchemy.orm import scoped_session, sessionmaker
import datetime
import numpy as np
from dateutil.parser import parse

In [2]:
if("notebooks" not in os.getcwd()):
    os.chdir("notebooks")
augurApp = augur.Application('../augur.config.json')
connect = augurApp.ghtorrentplus()
Piper, path= augurApp.piper()

In [3]:
#connect.db.execute("""DROP TABLE mailing_list_jobs""")

In [4]:
table_names = s.inspect(connect.db).get_table_names()
print(table_names)
val = False
mail_lists = True
numb = 0
if("mail_lists" not in table_names):
    mail_lists = False
if("mailing_list_jobs" in table_names):
    lists_createdSQL = s.sql.text("""SELECT project FROM mailing_list_jobs""")
    df1 = pd.read_sql(lists_createdSQL, connect.db)
    print(df1)
    val = True
    val = connect.db.engine.execute("""SELECT augurlistID FROM 
                                   mailing_list_jobs
                                   ORDER BY augurlistID DESC LIMIT 1""")
    for i in val:
        numb = i['augurlistID']
    
else:
    columns2 = "augurlistID","backend_name","mailing_list_url","project","last_message_date"
    df_mail_list = pd.DataFrame(columns=columns2)
    df_mail_list.to_sql(name="mailing_list_jobs",con=connect.db,if_exists='replace',index=False,
                        dtype={'augurlistID': s.types.Integer(),
                               'backend_name': s.types.VARCHAR(length=300),
                               'mailing_list_url': s.types.VARCHAR(length=300),
                               'project': s.types.VARCHAR(length=300),
                               'last_message_date': s.types.DateTime()
                        })
    lists_createdSQL = s.sql.text("""SELECT project FROM mailing_list_jobs""")
    df1 = pd.read_sql(lists_createdSQL, connect.db)
    connect.db.execute("ALTER TABLE mailing_list_jobs ADD PRIMARY KEY (augurlistID)")
    print("Created Table")
Base = declarative_base(connect.db)
class table(Base):
    __tablename__ = 'mailing_list_jobs'
    __table_args__={'autoload':True}
    
Session = sessionmaker(bind=connect.db)
session = Session()
res = session.query(table).all()

['github_issues', 'issue_response_time', 'mail_lists_sentiment_scores']
Created Table


In [5]:
def write_message(repo,type_archive,mail_check,pos,db,res,session,archives,numb,mail_lists,time=None):
    thread = None
    store = None
    k = 0
    di = {}
    #print("HEREEEE")
    if(pos == "lkml"):
        time = Piper.convert_date("Thu, 1 Jan 2013 20:37:11 +0000")
    for message in repo.fetch(from_date=time):
        #print(message,"\n\n\n\n\n\n\n\n")
        #print(message['data']['Message-ID'])
        if(type_archive == 'not_new'):
            mess_check = Piper.convert_date(message['data']['Date'])
            #mess_check = Piper.convert_date("Thu, 24 Mar 2019 20:37:11 +0000")
            #print(time)
        if(type_archive == 'not_new' and mess_check <= time ):
            print("Right here")
            continue            
        elif(type_archive == 'not_new' and mess_check > time):
            mail_check[pos] = 'update'
            
        ID = message['data']['Message-ID']
        try:
            message['data']['References']
            '''if(message['data']['Message-ID'] == '<CAFHD1sO814do11F9cKVZgr5fo+dw5q-VmfrYO_Q9vv6kXe8NjA@mail.gmail.com>'):
                print(thread)
                print(store)'''                             
            if((not thread == None) and (thread['data']['Message-ID'] not in message['data']['References'])):
                #bj = json.dumps(thread, indent=4, sort_keys=True)
                di[k] = thread
                #utfile.write(obj)
                #utfile.write('\n')
                store = None
                k+=1
                print("why")
                
            elif( (not store == None) and (store['data']['Message-ID'] not in message['data']['References'])):
                #print(message['data']['References'])
                di[k] = store
                #bj = json.dumps(store, indent=4, sort_keys=True)
                #utfile.write(obj)
                #utfile.write('\n')
                store = None
                print("yep")
                k+=1
            thread = message
        except:
            #print("got'em")
            if(not thread == None):
                di[k] = thread
                #bj = json.dumps(thread, indent=4, sort_keys=True)
                #utfile.write(obj)
                #utfile.write('\n')
                thread = None
                print("got-em")
                k+=1
            elif(not store == None):
                di[k] = store
                #bj = json.dumps(store, indent=4, sort_keys=True)
                #utfile.write(obj)
                #utfile.write('\n')
                store = None
                print("getting") 
                k+=1
            store = message
        if(len(di) == 5000):
            numb,mail_lists = Piper.make(connect.db,mail_check,archives,mail_lists,res,session,di,numb)
            di = {}
            k = 0
        #print("!"*50,"NEW MESSAGE","!"*50)
    if(len(di) < 5000 and len(di) > 0):
        print(len(di))
        #print(di)
        numb,mail_lists = Piper.make(connect.db,mail_check,archives,mail_lists,res,session,di,numb)
        di = {}
        k = 0
    else:
        di = {}
        k = 0
    if( (thread == None) and (store == None)):
        good = 1
    elif( (thread == None) and (not store == None) ):
        di[k] = store
        #obj = json.dumps(store, indent=4, sort_keys=True)
        #outfile.write(obj)
        #outfile.write('\n')
    elif( (store == None) and (not thread == None)):
        di[k] = thread
        #obj = json.dumps(thread, indent=4, sort_keys=True)
        #outfile.write(obj)
        #outfile.write('\n')
    elif(store['data']['Message-ID'] in thread['data']['References']):
        di[k] = thread
        #obj = json.dumps(thread, indent=4, sort_keys=True)
        #outfile.write(obj)
        #outfile.write('\n')
    else:
        di[k] = store
        #obj = json.dumps(store, indent=4, sort_keys=True)
        #outfile.write(obj)
        #outfile.write('\n')  
    #outfile.close()
    return numb,mail_lists
    

In [6]:
# create an Augur application so we can test our function
if "notebooks" in os.getcwd():
    os.chdir("..")
Piper, path= augurApp.piper()
print(path,"Place")
if(not os.path.exists(path)):
    file = open(path, "w+")
else:
    file = open(path, "r")
    print("yeah")
if (os.stat(path).st_size == 0):
    file.write("Link,mail_list\n")
    file.write("https://lists.opendaylight.org/pipermail/,\"aalldp-dev\"\n")
    file.write("https://lists.opendaylight.org/pipermail/,\"archetypes-dev\"\n")
    print("Please enter the mailing lists and the links for them please")
    print("Going to the default mailing lists")

count = 0
for line in file:
    print(line)
    count+=1
    if(count == 2):
        break
if(count == 2):
    #print(pd.read_csv(path))
    df = pd.read_csv(path)
    groups = df.groupby('Link').groups
    for group in groups:
        link = group
        mail_list = (df.loc[df['Link'] == group]['mail_list']).tolist()
        print(mail_list,"mail_list")            
        #link = "https://lists.opendaylight.org/pipermail/"
        #mail = ["aalldp-dev","alto-dev","archetypes-dev"]
        #mail = ["aalldp-dev","alto-dev","archetypes-dev","dev"]
        #mail = ["aalldp-dev","archetypes-dev","alto-dev"]
        #mail = ["aalldp-dev","archetypes-dev"]
        mail_check = {key:False for key in mail_list}
        print(mail_check)
        #print(os.getcwd())
        file = "mail_list"
        path = "/augur/data/archive-" 
        #numb = 0
        for x in range(len(mail_list)):
            #print(link+mail[x])
            if(mail_list[x] not in df1['project'].values ):
                mail_check[mail_list[x]] = 'new'
                #print(os.getcwd())
                #print(os.path.join(os.getcwd() + path+'.json'))
                place = os.path.join(os.getcwd() + path + mail_list[x] +'.json')           
                repo = Pipermail(url = link+ mail_list[x] + "/",dirpath="tmp/archives_"+mail_list[x])
                #print("Broken")
                #break
                #print(repo)
                outfile = open(place,"w+")
                numb,mail_lists = write_message(repo,'new',mail_check,mail_list[x],connect.db,res,session,\
                                     [mail_list[x]],numb,mail_lists)
                print("Created File",mail_list[x])
            else:
                last_updatedSQL = s.sql.text("""SELECT last_message_date FROM 
                mailing_list_jobs WHERE project = """ +  "'" + mail_list[x] + "'")
                last_updated_df = pd.read_sql(last_updatedSQL, connect.db)
                time = (last_updated_df['last_message_date'])  
                time = time.astype(object)
                place = os.path.join(os.getcwd() + path + 'temp_' + mail_list[x] +'.json')       
                repo = Pipermail(url = link+ mail_list[x] + "/",dirpath="tmp/archives_"+mail_list[x])
                outfile = open(place,"w+")
                print(time[0])
                print(type(time[0]))
                numb,mail_lists = write_message(repo,'not_new',mail_check,mail_list[x],connect.db,\
                              res,session,[mail_list[x]],numb,mail_lists,time[0])
                print("Checking to see for updated messages")
        print(mail_check)
        print("Finished downloading files")

2018-07-18 14:40:18 keanu-Inspiron-5567 perceval.backends.core.pipermail[19902] INFO Looking for messages from 'https://lists.opendaylight.org/pipermail/aalldp-dev/' since 1970-01-01 00:00:00+00:00
2018-07-18 14:40:18 keanu-Inspiron-5567 perceval.backends.core.pipermail[19902] INFO Downloading mboxes from 'https://lists.opendaylight.org/pipermail/aalldp-dev/' to since 1970-01-01 00:00:00+00:00


runtime/mailing_lists.csv Place
yeah
Link,mail_list

https://lists.opendaylight.org/pipermail/,"aalldp-dev"

['aalldp-dev', 'archetypes-dev'] mail_list
{'aalldp-dev': False, 'archetypes-dev': False}


2018-07-18 14:40:21 keanu-Inspiron-5567 perceval.backends.core.pipermail[19902] INFO 2/2 MBoxes downloaded
2018-07-18 14:40:21 keanu-Inspiron-5567 perceval.backends.core.mbox[19902] INFO Done. 6/6 messages fetched; 0 ignored
2018-07-18 14:40:21 keanu-Inspiron-5567 perceval.backends.core.pipermail[19902] INFO Fetch process completed


getting
getting
yep
got-em
4
['aalldp-dev']
2018-07-06 18:39:58
File uploaded  13


2018-07-18 14:40:31 keanu-Inspiron-5567 perceval.backends.core.pipermail[19902] INFO Looking for messages from 'https://lists.opendaylight.org/pipermail/archetypes-dev/' since 1970-01-01 00:00:00+00:00
2018-07-18 14:40:31 keanu-Inspiron-5567 perceval.backends.core.pipermail[19902] INFO Downloading mboxes from 'https://lists.opendaylight.org/pipermail/archetypes-dev/' to since 1970-01-01 00:00:00+00:00


...

getting
1
['archetypes-dev']
File uploaded  4


2018-07-18 14:40:44 keanu-Inspiron-5567 perceval.backends.core.pipermail[19902] INFO Looking for messages from 'https://lore.kernel.org/lkml/' since 2013-01-01 20:37:11
2018-07-18 14:40:44 keanu-Inspiron-5567 perceval.backends.core.pipermail[19902] INFO Downloading mboxes from 'https://lore.kernel.org/lkml/' to since 2013-01-01 20:37:11


Mailing List Job uploaded
Finished
Created File archetypes-dev
{'aalldp-dev': 'new', 'archetypes-dev': 'new'}
Finished downloading files
['lkml'] mail_list
{'lkml': False}


2018-07-18 14:43:17 keanu-Inspiron-5567 perceval.backends.core.pipermail[19902] INFO 92/92 MBoxes downloaded
2018-07-18 14:43:17 keanu-Inspiron-5567 perceval.backends.core.mbox[19902] INFO Done. 2/2 messages fetched; 0 ignored
2018-07-18 14:43:17 keanu-Inspiron-5567 perceval.backends.core.pipermail[19902] INFO Fetch process completed


getting
1
['lkml']
File uploaded  4
Mailing List Job uploaded
Finished
Created File lkml
{'lkml': 'new'}
Finished downloading files


In [7]:
'''Base = declarative_base(connect.db)
class table(Base):
    __tablename__ = 'mailing_list_jobs'
    __table_args__={'autoload':True}
    
Session = sessionmaker(bind=connect.db)
session = Session()
res = session.query(table).all()

Piper.make(connect.db,mail_check,mail_list,mail_lists,res,session)'''

"Base = declarative_base(connect.db)\nclass table(Base):\n    __tablename__ = 'mailing_list_jobs'\n    __table_args__={'autoload':True}\n    \nSession = sessionmaker(bind=connect.db)\nsession = Session()\nres = session.query(table).all()\n\nPiper.make(connect.db,mail_check,mail_list,mail_lists,res,session)"

In [8]:
#print(mail_lists)

In [37]:
repo = Pipermail(url="https://kernel.googlesource.com/pub/scm/public-inbox/vger.kernel.org/lkml/0",\
                 dirpath="tmp/archives/temp1")
di = {}
for message in repo.fetch(from_date = Piper.convert_date("Thu, 1 Jul 2018 20:37:11 +0000")):
    print(message)
    print("\n\n\n")

2018-07-18 15:45:08 keanu-Inspiron-5567 perceval.backends.core.pipermail[19902] INFO Looking for messages from 'https://kernel.googlesource.com/pub/scm/public-inbox/vger.kernel.org/lkml/0' since 2018-07-01 20:37:11
2018-07-18 15:45:08 keanu-Inspiron-5567 perceval.backends.core.pipermail[19902] INFO Downloading mboxes from 'https://kernel.googlesource.com/pub/scm/public-inbox/vger.kernel.org/lkml/0' to since 2018-07-01 20:37:11
2018-07-18 15:45:09 keanu-Inspiron-5567 perceval.backends.core.pipermail[19902] INFO 0/0 MBoxes downloaded
2018-07-18 15:45:09 keanu-Inspiron-5567 perceval.backends.core.mbox[19902] INFO Done. 0/0 messages fetched; 0 ignored
2018-07-18 15:45:09 keanu-Inspiron-5567 perceval.backends.core.pipermail[19902] INFO Fetch process completed


In [10]:
#di[0]['data']['Date']

In [11]:
'''dict = {}
di1 = {0:2}
dict[0] = 0
print(dict)
dict['b'] = 1
print(dict)
dict['d'] = di1
print(dict)
print(dict[0])
print(len(dict))'''

"dict = {}\ndi1 = {0:2}\ndict[0] = 0\nprint(dict)\ndict['b'] = 1\nprint(dict)\ndict['d'] = di1\nprint(dict)\nprint(dict[0])\nprint(len(dict))"